In [1]:

KEY_TEXT=4
MSG_TEXT=4
C_TEXT=4
R_LEN=2*KEY_TEXT
LEARNING_RATE=0.1
LAMBDA=5
BATCH_SIZE=512
m_train=2**(MSG_TEXT)
NUM_BATCH_SIZE=128
MAX_INTERATION=100000
EVE_ONE_BIT_WRONG_THRESH = 0.97
BOB_ONE_BIT_WRONG_THRESH=0.0025
CLIP_VALUE = 1
LOOPS_PER_PRINT = 100   # every 100 loops print one time
CYCLE_ALICE_BOB=5
CYCLE_EVE=10


In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
def function(x):
  return torch.acos((1-2*x))
  ''' ''' 

def inv_function(y):
    
  return (1-torch.cos(y))/2

In [4]:
class Network(nn.Module):
    '''
    Standard Convolutional layers setup used by Alice and Bob.
    Input: 2N tensor, PlainText+MessageKey(Alice) or CipherText+Key(Bob)
    Output: N tensor, CipherText(Alice) or PlainText(Bob)
    '''
    def __init__(self):
        super(Network,self).__init__()
        self.fc_in =MSG_TEXT + KEY_TEXT
        self.fc_out = KEY_TEXT
        self.l1 = nn.Linear(self.fc_in,self.fc_out,bias=False)

    def forward(self,input):

        inputs = function(input)
        #print("inputs:",inputs)
        out = self.l1(inputs) #requires_grad
        #print("out:",out.requires_grad)
        _out = inv_function(out) #possibly removes requires grad
        #print("_out:",_out.requires_grad)
        #print("out:",_out)
        
        output = _out
        #print('one pass AB', output.size())
        return output


In [5]:
class Network2(nn.Module):
    '''
    Standard Convolutional layers setup used by Alice and Bob.
    Input: 2N tensor, PlainText+MessageKey(Alice) or CipherText+Key(Bob)
    Output: N tensor, CipherText(Alice) or PlainText(Bob)
    '''
    def __init__(self):
        super(Network2,self).__init__()
        self.fc_in =MSG_TEXT+MSG_TEXT+C_TEXT
        self.fc_out = C_TEXT
        self.fc_sec=R_LEN
        self.fc_sort = 2
        self.l1 = nn.Linear(self.fc_in,self.fc_sec,bias=False)
        self.l2 = nn.Linear(self.fc_sec,self.fc_sort,bias=False)

    def forward(self,input):

        inputs = function(input)
        #print("inputs:",inputs)
        out = self.l1(inputs) #requires_grad
        #print("out:",out.requires_grad
        _out = inv_function(out) #possibly removes requires grad
        out1 = self.l2(_out)
        
        x = F.softmax(out1,dim=1)
        
        #print("_out:",_out.requires_grad)
        #print("out:",_out)
        
        output = x
        #print('one pass AB', output.size())
        return output


In [6]:
def generate_data(batch_size, ptext_size, key_size):
    """
    generate data.
    :param batch_size: batch size, hyper-parameters, in this program BATCH_SIZE is 256
    :param ptext_size: ptext size, hyper-parameters, in this program PTEXT_SIZE is 16
    :param key_size: key's size, hyper-parameters, in this program KEY_SIZE is 16
    :return: ptext and key, in this program size are both [256, 16]
    """
    ptext = torch.randint(0, 2, (batch_size, ptext_size), dtype=torch.float) * 1
    key = torch.randint(0, 2, (batch_size, key_size), dtype=torch.float) * 1
    return ptext, key

In [7]:
def gen_data(n=BATCH_SIZE, msg_len=MSG_TEXT, key_len=KEY_TEXT):
    return (np.random.randint(0, 2, size=(n, msg_len)) * 1), \
           (np.random.randint(0, 2, size=(n, key_len)) * 1), \
           (np.random.randint(0, 2, size=(n, key_len)) * 1)

In [8]:
def plot_wrong( alice_bob_wrong_for_plot,eve_wrong_for_plot):
    """
    plot epoch-wrong picture
    
    :param alice_bob_wrong_for_plot: a list, element is the mean of bob one bit wrong
    :return:
    """
    plt.plot(range(1, len(eve_wrong_for_plot)+1), eve_wrong_for_plot, label='eve one bit wrong mean')
    plt.plot(range(1, len(alice_bob_wrong_for_plot)+1), alice_bob_wrong_for_plot, label='bob one bit wrong mean')
    plt.xlabel("Epochs")
    plt.ylabel("One Bit Wrong")
    plt.title("optimizer_alice_bob_times")
    plt.legend()
    plt.show()


def plot_errors(eve_errors,alice_bob_loss):
  sns.set_style("darkgrid")
  #plt.plot(bob_errors)
  plt.plot(eve_errors)
  plt.plot(alice_bob_loss)
  plt.legend(['bob', 'eve','alice bob'])
  plt.xlabel('Epoch')
  plt.ylabel('Lowest Decryption error achieved')
  plt.show()


In [9]:
alice_bob = Network()
eve = Network2()
data = gen_data(BATCH_SIZE, MSG_TEXT, KEY_TEXT)
msg = torch.tensor(data[0], dtype=torch.float)
msg1 = torch.tensor(data[1], dtype=torch.float)
key = torch.tensor(data[2], dtype=torch.float)
alice_input = torch.cat((msg, key), 1)
inputs = torch.cat((msg, msg1), 1)  

 


In [10]:
ctext= alice_bob(alice_input)
      


In [11]:
ctext[0]

tensor([0.0646, 0.0926, 0.0096, 0.0253], grad_fn=<SelectBackward>)

In [12]:
bob_input = torch.cat((ctext, key), 1)
ptext_bob = alice_bob(bob_input)


In [13]:
ptext_bob[0]

tensor([0.0028, 0.0457, 0.0031, 0.1087], grad_fn=<SelectBackward>)

In [30]:
eve_input = torch.cat((ctext, inputs), dim=1)
ptext_eve= eve(eve_input)

In [31]:
ptext_eve[0]

tensor([0.4989, 0.5011], grad_fn=<SelectBackward>)

In [29]:
p1=[]
p2=[]
pi0=[]
pi1=[]
for i in range(len(ptext_eve)):
    if ptext_eve[i][0]> ptext_eve[i][1]:
        p1.append(0)
        pi0.append(ptext_eve[i][0])
    else:
        p2.append(1)
        pi1.append(ptext_eve[i][1])
#print(pi1)
    


In [18]:
pi0=torch.tensor(pi0).numpy()
pi1=torch.tensor(pi1).numpy()
#pi=torch.cat((pi0,pi1),dim=1)
p1=torch.tensor(p1).numpy()
p2=torch.tensor(p2).numpy()
#P=p1+p2

In [19]:
vect=ptext_eve[:,1]>ptext_eve[:,0]

In [28]:
vec= vect.type(torch.ByteTensor)

In [32]:
vec[0]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         /.,m

tensor(1, dtype=torch.uint8)

In [ ]:
def train():

  alice_bob_one_bit_wrong_mean = 0.0
  

  alice_bob_wrong_for_plot = []
  eve_wrong_for_plot = []
  torch.manual_seed(10)
  alice_bob = Network()
  eve = Network2()

  alice_bob.train()
  #bob.train()

  optimizer_alice_bob = optim.Adam(alice_bob.parameters(), lr=LEARNING_RATE)
  optimizer_eve = optim.Adam(eve.parameters(), lr=LEARNING_RATE)


  alice_bob_reconstruction_error = nn.L1Loss()
  eve_reconstruction_error = nn.CrossEntropyLoss()

  for i in range(30):

    start_time = time.time()

    '''torch.save({
        'Alice_state_dict': alice_bob.state_dict(),
        
        'optimizer_alice_bob_state_dict': optimizer_alice_bob.state_dict(),
         }, SAVE_PATH)'''
    checkpoint={'alice_bob':Network(),
                'state_dict': alice_bob.state_dict(),
                'optimizer': optimizer_alice_bob.state_dict()
                }
    torch.save(checkpoint,'checkpoint.pth')

    #for network,num_minibatch in {'alice_bob': NUM_BATCH_SIZE }.items():

    for minibatch in range(128):
      np.random.seed(1)
      data = gen_data(BATCH_SIZE, MSG_TEXT, KEY_TEXT)
      msg = torch.tensor(data[0], dtype=torch.float)
      msg1 = torch.tensor(data[1], dtype=torch.float)
      key = torch.tensor(data[2], dtype=torch.float)
      alice_input = torch.cat((msg, key), 1)
      inputs = torch.cat((msg, msg1), 1)  
      optimizer_alice_bob.zero_grad()
      optimizer_eve.zero_grad()
      with torch.no_grad():
        ctext= alice_bob(alice_input)
      
      bob_input = torch.cat((ctext, key), 1)
      ptext_bob = alice_bob(bob_input)
      eve_input = torch.cat((ctext, inputs), 1)
      ptext_eve = eve(eve_input)
      
      #print(ptext_bob)

      alice_bob_error=alice_bob_reconstruction_error(ptext_bob,msg)
      eve_error =eve_reconstruction_error(ptext_eve,msg)
      alice_bob_loss=alice_bob_error -  5*min(eve_error.item(),0.5)
      



      
      #optimizer_alice_bob.zero_grad()
      
      alice_bob_loss.backward(retain_graph=True)
      
      
      #print(alice_bob_error.grad)
      #print(alice_bob_error)
      #nn.utils.clip_grad_value_(alice_bob.parameters(), CLIP_VALUE)
      #nn.utils.clip_grad_value_(bob.parameters(), CLIP_VALUE)


      #optimizer_alice.step()
      optimizer_alice_bob.step()
      eve_error.backward()
      optimizer_eve.step()
        

      time_elapsed = time.time() - start_time
    print(alice_bob_error,eve_error)
    bob_one_bit_wrong_mean = alice_bob_error.cpu().detach().numpy()
    eve_ = eve_error.cpu().detach().numpy()
    


    '''if i % LOOPS_PER_PRINT == 0:
      print(f'Epoch: {i + 1:06d} | '
            f'one epoch time: {time_elapsed:.3f} | '
            f'bob one bit wrong: {bob_one_bit_wrong_mean:.4f} |')
    alice_bob_wrong_for_plot.append(bob_one_bit_wrong_mean)'''
    alice_bob_wrong_for_plot.append(bob_one_bit_wrong_mean)
    eve_wrong_for_plot.append(eve_)

  plot_wrong(alice_bob_wrong_for_plot,eve_wrong_for_plot)
  checkpoint={'alice_bob':Network(),
                'state_dict': alice_bob.state_dict(),
                'optimizer': optimizer_alice_bob.state_dict()
                }
  torch.save(checkpoint,'checkpoint1v.pth')
  checkpoint1={'eve': Network2(),
                'state_dict1': eve.state_dict(),
                'optimizer1': optimizer_eve.state_dict()
                }
  torch.save(checkpoint1,'checkpoint11v.pth')

if __name__ == "__main__":

    train()